In [6]:
import os
import matplotlib.pyplot as plt
import json

import sys
sys.path.insert(0, '../')
from forest3D import lidar_IO,treeDetector,ground_removal,utilities,detection_tools
from forest3D.pcd import read_pcd
# size limitation: 120m x 120m

output_dir = '../../data/outputs'

# read data
path = "../../data/pcd_files/150_2-160_2_slam.pcd"
pcd_file = read_pcd(path)
xyz_data = pcd_file['points'][['x','y','z']].values
offset = [0,0,0]


# remove ground, output pcd to help with labelling
offset = [0,0,0]
xyz_data_gr = ground_removal.removeGround(xyz_data, offset, thresh=2.0,proc_path=output_dir, name='')
ground_pts = ground_removal.load_ground_surface(os.path.join(output_dir, '_ground_surface.ply'))


# configure raster
config_dict = {'raster_layers': ['vertical_density', 'canopy_height'], 'support_window': [11, 7, 1],
               'normalisation': 'rescale+histeq','doHisteq':[False,True],'res': 0.1, 'gridSize': [600, 600, 1000]}






# GEN

In [7]:
MAX_RETURN_INTENSITY = 77.0
xyz_data[:, 3] /= MAX_RETURN_INTENSITY

# remove ground, output pcd to help with labelling
offset = [0,0,0]
xyz_gr,colour_gr = ground_removal.removeGround(xyz_data[:,:3], returns=xyz_data[:,3], offset=offset, thresh=2.0,
                                               proc_path=output_dir, name='')
ground_pts = ground_removal.load_ground_surface(os.path.join(output_dir, '_ground_surface.ply'))
xyz_colour_gr = np.hstack((xyz_gr,colour_gr))




x_centre = 20
y_centre = 20
radius = 30

# extract pointcloud crop
xyz_crop_gr = detection_tools.circle_crop(xyz_colour_gr,x_centre,y_centre,radius)

lidar_IO.writePly(os.path.join(output_dir, 'detection_%i.ply'%i), xyz_crop_gr, offset)




IndexError: index 3 is out of bounds for axis 1 with size 3

# HVP

In [3]:
rasterMaker = treeDetector.RasterDetector(**config_dict)

for i in range(1,20):

    # read in training crop info from csv
    path = '../../data/outputs/training_stems_%03d.csv'%i
    plot_details = utilities.read_csv(path,header_rows=[0])
    x_centre = float(plot_details['header'][1])
    y_centre = float(plot_details['header'][2])
    radius = float(plot_details['header'][3])
    gt_coords = plot_details['data'][:,1:3]

    # extract pointcloud crop
    xyz_crop_gr = detection_tools.circle_crop(xyz_data_gr,x_centre,y_centre,radius)


    # create raster
    raster = rasterMaker.rasterise(xyz_crop_gr,ground_pts=ground_pts)


    # save rasters
    filename = os.path.split(path)[1].split('.')[0]
    plt.imsave(os.path.join(output_dir,'raster_'+filename+'.jpg'), raster)

with open(os.path.join(output_dir,'raster_config.json'), 'w') as outfile:
    json.dump(config_dict, outfile)

FileNotFoundError: [Errno 2] No such file or directory: '../../data/outputs/training_stems_001.csv'

# Caraboast

In [4]:
import os
import matplotlib.pyplot as plt
import json

import sys
sys.path.insert(0, '../')
from forest3D import lidar_IO,treeDetector,ground_removal


# size limitation: 120m x 120m

output_dir = '../../data/outputs'

plot_list = ['Site2_Plot_0_p_60m_fixed', 'Site2_Plot_1_p_60m_fixed', 'Site4_Plot_04_p_60m_fixed', 'Site4_Plot_05_p_60m_fixed',
             'Site4_Plot_6_p_60m_fixed', 'Site8_Plot_15_p_60m_fixed', 'Site8_Plot_16_p_60m_fixed', 'Site9_Plot_17_p_60m_fixed']


# configure raster
config_dict = {'raster_layers': ['vertical_density', 'canopy_height'], 'support_window': [1, 1, 1],
               'normalisation': 'rescale+histeq','doHisteq':[True,True],'res': 0.2, 'gridSize': [600, 600, 1000]}
rasterMaker = treeDetector.RasterDetector(**config_dict)


xyz_clr_data = xyz_data
slices = detection_tools.sliding_window_3d(xyz_data, stepSize=stepSize,windowSize=windowSize)

for i in range(8):

    offset = [0,0,0]

    # remove ground, output pcd to help with labelling
    xyz_data_gr = ground_removal.removeGround(xyz_data,offset,thresh=2.0,
                                                         proc_path=output_dir, name='')
    ground_pts = ground_removal.load_ground_surface(os.path.join(output_dir,'_ground_surface.ply'))


    # create raster
    raster = rasterMaker.rasterise(xyz_data_gr,ground_pts=ground_pts)


    # save rasters
    filename = os.path.split(path)[1].split('.')[0]
    plt.imsave(os.path.join(output_dir,'raster_'+filename+'.jpg'), raster)

with open(os.path.join(output_dir,'raster_config.json'), 'w') as outfile:
    json.dump(config_dict, outfile)





In [10]:
raster.shape

(600, 600, 3)